In [1]:
pip install astropy pymongo pyspark matplotlib pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pymongo
import os
import pandas as pd
from pyspark.sql import SparkSession
from astropy.io import fits
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Configuración de la base de datos MongoDB
def setup_database():
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["wmap_database"]
    return db

# Guardar datos de imagen en fragmentos pequeños
def split_and_insert_image_data(db, image_data, hdu_index):
    collection = db["image_data"]
    chunk_size = 1000  # Filas por fragmento (ajusta según sea necesario)
    
    for i in range(0, len(image_data), chunk_size):
        chunk = image_data[i:i + chunk_size]
        document = {
            "type": "ImageHDU",
            "hdu_index": hdu_index,
            "chunk_index": i // chunk_size,
            "data": chunk.tolist()
        }
        collection.insert_one(document)

# Guardar matrices grandes como archivos externos y registrar referencias en MongoDB
def save_large_data_to_file(db, image_data, file_name, hdu_index):
    os.makedirs("output", exist_ok=True)
    file_path = f"output/{file_name}_hdu_{hdu_index}.npy"
    np.save(file_path, image_data)  # Guardar como archivo .npy
    
    # Registrar referencia en MongoDB
    db["image_references"].insert_one({
        "type": "ImageHDU",
        "hdu_index": hdu_index,
        "file_path": file_path
    })

# Procesar y almacenar tablas en MongoDB
def process_table_data(db, table_data, hdu_index):
    collection = db["table_data"]
    column_names = table_data.columns.names
    
    for row in table_data:
        document = {
            key: row[key].item() if isinstance(row[key], np.generic) else row[key]
            for key in column_names
        }
        document["hdu_index"] = hdu_index
        collection.insert_one(document)

# Procesar archivo FITS
def process_fits_file(db, fits_file):
    with fits.open(fits_file) as hdul:
        print(f"Procesando archivo: {fits_file}")
        hdul.info()
        
        for i, hdu in enumerate(hdul):
            if isinstance(hdu, fits.ImageHDU):
                print(f"Procesando ImageHDU en extensión {i}")
                image_data = hdu.data
                if image_data is not None:
                    # Elige entre dividir en fragmentos o guardar en un archivo
                    if image_data.nbytes > 16 * 1024 * 1024:  # Si supera 16 MB
                        save_large_data_to_file(db, image_data, "image_data", i)
                    else:
                        split_and_insert_image_data(db, image_data, i)
            elif isinstance(hdu, fits.BinTableHDU):
                print(f"Procesando BinTableHDU en extensión {i}")
                process_table_data(db, hdu.data, i)


# Consultas avanzadas en MongoDB
def advanced_queries(db):
    print("Consultas avanzadas en la base de datos:")
    # Promedio de temperatura
    avg_temp = db["table_data"].aggregate([
        {"$group": {"_id": None, "TEMPERATURE": {"$avg": "$TEMPERATURE"}}}
    ])
    print("Promedio de temperatura:", list(avg_temp))
    
    # Anomalías térmicas
    anomalies = db["table_data"].find({"TEMPERATURE": {"$gt": 50}})
    print("Anomalías térmicas:", list(anomalies))
    
    # Agrupación por frecuencia
    freq_groups = db["table_data"].aggregate([
        {"$group": {"_id": "$FREQUENCY", "count": {"$sum": 1}}}
    ])
    print("Agrupación por frecuencia:", list(freq_groups))

# Main function
if __name__ == "__main__":
    # Configuración de la base de datos
    db = setup_database()
    
    # Archivos FITS de ejemplo (Reemplaza con tus archivos reales)
    fits_files = ["wmap_mollweide_band_imap_r9_9yr_K_v5.fits"]  # Cambia a las rutas reales
    
    # Procesar archivos FITS
    for file in fits_files:
        process_fits_file(db, file)
    
    # Realizar consultas avanzadas
    advanced_queries(db)


Procesando archivo: wmap_mollweide_band_imap_r9_9yr_K_v5.fits
Filename: wmap_mollweide_band_imap_r9_9yr_K_v5.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      22   ()      
  1  TEMPERATURE    1 ImageHDU        22   (4096, 2048)   float32   
  2  N_OBS         1 ImageHDU        22   (4096, 2048)   float32   
Procesando ImageHDU en extensión 1
Procesando ImageHDU en extensión 2
Consultas avanzadas en la base de datos:
Promedio de temperatura: []
Anomalías térmicas: []
Agrupación por frecuencia: []


In [8]:
def generate_simulated_cmb_data(db_products):

    for category in db_products:
        if category['category'] == 'Temperatura':
            url = category['products'][0]['url']

            try:
                with fits.open(url) as hdul:
                    data = hdul[1].data
                    simulated_data = data['TEMPERATURE']

                    nside = hp.get_nside(simulated_data)
                    npix = hp.nside2npix(nside)

                    simulated_data = simulated_data.reshape(npix)

                    lon = np.linspace(-180, 180, 360)
                    lat = np.linspace(-90, 90, 180)
                    lon_grid, lat_grid = np.meshgrid(lon, lat)

                    simulated_data_2d = hp.get_interp_val(
                        simulated_data, np.radians(lon_grid), np.radians(lat_grid)
                    )

                    return simulated_data_2d

            except Exception as e:
                return None


    return None



def plot_mollweide_map(data):
    """
    Genera un mapa Mollweide con los datos proporcionados.
    """
    plt.figure(figsize=(10, 6))
    ax = plt.subplot(111, projection="mollweide")


    n_lat, n_lon = data.shape
    lon = np.linspace(-np.pi, np.pi, n_lon)
    lat = np.linspace(-np.pi / 2, np.pi / 2, n_lat)
    lon_grid, lat_grid = np.meshgrid(lon, lat)


    im = ax.pcolormesh(lon_grid, lat_grid, data, cmap="inferno", shading="auto")
    cbar = plt.colorbar(im, ax=ax, orientation="horizontal", fraction=0.05, pad=0.1)
    cbar.set_label("Temperatura (μK)(10^-6 K)")
    plt.title("Mapa del CMB en base a la temperatura de los datos")
    plt.show()



simulated_cmb_data = generate_simulated_cmb_data(db)


if simulated_cmb_data is not None:
    plot_mollweide_map(simulated_cmb_data)

TypeError: 'Database' object is not iterable